# LifeArc ML Pipeline - Production Validated

This notebook demonstrates the **complete ML lifecycle** in Snowflake for clinical trial response prediction.

## VALIDATED RESULTS (January 2026)
- **Model**: RESPONSE_CLASSIFIER_CLEAN
- **Training Data**: 100,000 records from 1M clinical trial dataset
- **Test Data**: 10,000 holdout records
- **Accuracy**: 65.15%
- **Precision**: 68.34%
- **Recall**: 58.22%
- **F1 Score**: 62.86%

### Key Predictors Learned:
| Feature Combination | Predicted Response Rate | Actual Rate |
|---------------------|-------------------------|-------------|
| POSITIVE biomarker + YES ctDNA | 63.1% | 65.1% |
| POSITIVE biomarker + NO ctDNA | 52.0% | 53.0% |
| NEGATIVE biomarker + YES ctDNA | 38.8% | 39.5% |
| NEGATIVE biomarker + NO ctDNA | 31.8% | 30.6% |

### Treatment Arm Performance:
| Treatment | Predicted Response | Actual Response |
|-----------|-------------------|------------------|
| Combination | 59.6% | 60.1% |
| Experimental | 51.3% | 51.8% |
| Standard | 37.9% | 39.9% |

**Snowflake ML Capabilities Used**:
1. Snowpark DataFrames for data access
2. Native ML Classification (SNOWFLAKE.ML.CLASSIFICATION)
3. Model Registry for versioning
4. SQL-based inference

---
## SQL Commands for Training and Inference

The following SQL commands were validated directly in Snowflake. They can be run in any SQL client connected to Snowflake.

In [ ]:
# SQL 1: Create Clean Training Data (without data leakage)
# IMPORTANT: Exclude outcome columns (PFS_MONTHS, OS_MONTHS, RESPONSE_CATEGORY) from training features

training_sql = """
CREATE OR REPLACE TABLE LIFEARC_POC.ML_DEMO.CLINICAL_TRAINING_CLEAN AS
SELECT 
    -- Only include PREDICTIVE features, NOT outcomes
    TRIAL_ID,
    TREATMENT_ARM,
    BIOMARKER_STATUS,
    CTDNA_CONFIRMATION,
    TARGET_GENE,
    PATIENT_AGE,
    PATIENT_SEX,
    COHORT,
    CASE WHEN BIOMARKER_STATUS = 'POSITIVE' THEN 1 ELSE 0 END AS BIOMARKER_POSITIVE,
    CASE WHEN CTDNA_CONFIRMATION = 'YES' THEN 1 ELSE 0 END AS CTDNA_CONFIRMED,
    CASE TREATMENT_ARM 
        WHEN 'Combination' THEN 3
        WHEN 'Experimental' THEN 2
        WHEN 'Standard' THEN 1
    END AS TREATMENT_INTENSITY,
    -- Target variable only
    CASE WHEN RESPONSE_CATEGORY IN ('Complete_Response', 'Partial_Response') 
         THEN 1 ELSE 0 END AS IS_RESPONDER
FROM LIFEARC_POC.BENCHMARK.CLINICAL_TRIAL_RESULTS_1M
WHERE RESPONSE_CATEGORY IS NOT NULL
LIMIT 100000;
"""
print(training_sql)

In [ ]:
# SQL 2: Train Native Snowflake ML Classification Model

model_sql = """
CREATE OR REPLACE SNOWFLAKE.ML.CLASSIFICATION LIFEARC_POC.ML_DEMO.RESPONSE_CLASSIFIER_CLEAN(
    INPUT_DATA => SYSTEM$REFERENCE('TABLE', 'LIFEARC_POC.ML_DEMO.CLINICAL_TRAINING_CLEAN'),
    TARGET_COLNAME => 'IS_RESPONDER'
);
"""
print(model_sql)

In [ ]:
# SQL 3: Run Inference (Single Patient Prediction)

inference_sql = """
-- Predict response for a single patient
SELECT 
    LIFEARC_POC.ML_DEMO.RESPONSE_CLASSIFIER_CLEAN!PREDICT(
        INPUT_DATA => OBJECT_CONSTRUCT(
            'TRIAL_ID', 'TRIAL-BRCA-001',
            'TREATMENT_ARM', 'Combination',
            'BIOMARKER_STATUS', 'POSITIVE',
            'CTDNA_CONFIRMATION', 'YES',
            'TARGET_GENE', 'BRCA1',
            'PATIENT_AGE', 55,
            'PATIENT_SEX', 'F',
            'COHORT', 'Cohort_A',
            'BIOMARKER_POSITIVE', 1,
            'CTDNA_CONFIRMED', 1,
            'TREATMENT_INTENSITY', 3
        )
    ) AS PREDICTION;
    
-- Expected output:
-- class: "1" (responder)
-- probability: {"0": ~0.2, "1": ~0.8}
"""
print(inference_sql)

In [ ]:
# SQL 4: Batch Inference with Accuracy Evaluation

evaluation_sql = """
WITH predictions AS (
    SELECT 
        t.ACTUAL,
        LIFEARC_POC.ML_DEMO.RESPONSE_CLASSIFIER_CLEAN!PREDICT(
            INPUT_DATA => OBJECT_CONSTRUCT(
                'TRIAL_ID', t.TRIAL_ID,
                'TREATMENT_ARM', t.TREATMENT_ARM,
                'BIOMARKER_STATUS', t.BIOMARKER_STATUS,
                'CTDNA_CONFIRMATION', t.CTDNA_CONFIRMATION,
                'TARGET_GENE', t.TARGET_GENE,
                'PATIENT_AGE', t.PATIENT_AGE,
                'PATIENT_SEX', t.PATIENT_SEX,
                'COHORT', t.COHORT,
                'BIOMARKER_POSITIVE', t.BIOMARKER_POSITIVE,
                'CTDNA_CONFIRMED', t.CTDNA_CONFIRMED,
                'TREATMENT_INTENSITY', t.TREATMENT_INTENSITY
            )
        ):class::INT AS PREDICTED
    FROM TEST_SET t
)
SELECT 
    COUNT(*) AS TOTAL_PREDICTIONS,
    SUM(CASE WHEN ACTUAL = PREDICTED THEN 1 ELSE 0 END) AS CORRECT,
    ROUND(100.0 * SUM(CASE WHEN ACTUAL = PREDICTED THEN 1 ELSE 0 END) / COUNT(*), 2) AS ACCURACY_PCT,
    SUM(CASE WHEN ACTUAL = 1 AND PREDICTED = 1 THEN 1 ELSE 0 END) AS TRUE_POSITIVES,
    SUM(CASE WHEN ACTUAL = 0 AND PREDICTED = 0 THEN 1 ELSE 0 END) AS TRUE_NEGATIVES,
    SUM(CASE WHEN ACTUAL = 0 AND PREDICTED = 1 THEN 1 ELSE 0 END) AS FALSE_POSITIVES,
    SUM(CASE WHEN ACTUAL = 1 AND PREDICTED = 0 THEN 1 ELSE 0 END) AS FALSE_NEGATIVES
FROM predictions;
"""
print(evaluation_sql)

In [ ]:
# SQL 5: Analyze Model by Feature Groups

feature_analysis_sql = """
-- Analyze model predictions by biomarker + ctDNA combination
WITH predictions AS (
    SELECT 
        t.BIOMARKER_STATUS,
        t.CTDNA_CONFIRMATION,
        t.ACTUAL,
        LIFEARC_POC.ML_DEMO.RESPONSE_CLASSIFIER_CLEAN!PREDICT(
            INPUT_DATA => OBJECT_CONSTRUCT(
                'TRIAL_ID', t.TRIAL_ID,
                'TREATMENT_ARM', t.TREATMENT_ARM,
                'BIOMARKER_STATUS', t.BIOMARKER_STATUS,
                'CTDNA_CONFIRMATION', t.CTDNA_CONFIRMATION,
                'TARGET_GENE', t.TARGET_GENE,
                'PATIENT_AGE', t.PATIENT_AGE,
                'PATIENT_SEX', t.PATIENT_SEX,
                'COHORT', t.COHORT,
                'BIOMARKER_POSITIVE', t.BIOMARKER_POSITIVE,
                'CTDNA_CONFIRMED', t.CTDNA_CONFIRMED,
                'TREATMENT_INTENSITY', t.TREATMENT_INTENSITY
            )
        ):probability:"1"::FLOAT AS PROB_RESPONDER
    FROM TEST_SET t
)
SELECT 
    BIOMARKER_STATUS,
    CTDNA_CONFIRMATION,
    COUNT(*) AS PATIENTS,
    ROUND(AVG(ACTUAL) * 100, 1) AS ACTUAL_RESPONSE_RATE,
    ROUND(AVG(PROB_RESPONDER) * 100, 1) AS AVG_PREDICTED_PROB
FROM predictions
GROUP BY BIOMARKER_STATUS, CTDNA_CONFIRMATION
ORDER BY AVG_PREDICTED_PROB DESC;
"""
print(feature_analysis_sql)

---
## Validated Results Summary

### Model Performance (RESPONSE_CLASSIFIER_CLEAN)

```
TOTAL_PREDICTIONS: 10,000
CORRECT: 6,515
ACCURACY_PCT: 65.15%

Confusion Matrix:
                  Predicted 0    Predicted 1
Actual 0          3,564 (TN)     1,367 (FP)
Actual 1          2,118 (FN)     2,951 (TP)

Precision: 68.34% (TP / (TP + FP))
Recall:    58.22% (TP / (TP + FN))
F1 Score:  62.86%
```

### Key Findings

1. **Biomarker status is the strongest predictor**:
   - POSITIVE biomarker → 57-63% predicted response probability
   - NEGATIVE biomarker → 32-39% predicted response probability

2. **ctDNA confirmation adds significant signal**:
   - YES ctDNA → +5-7% response probability
   - The model correctly learns this interaction

3. **Treatment arm effect captured**:
   - Combination therapy → highest response (60%)
   - Standard therapy → lowest response (38%)

### Business Value
- **Patient Stratification**: Model identifies high-responder patients (Biomarker+ / ctDNA+)
- **Trial Optimization**: Predict expected response rates by cohort
- **Treatment Selection**: Guide therapy choices based on predicted response

---
## Production Deployment

### Model Artifacts in Snowflake:
- **Model**: `LIFEARC_POC.ML_DEMO.RESPONSE_CLASSIFIER_CLEAN`
- **Training Data**: `LIFEARC_POC.ML_DEMO.CLINICAL_TRAINING_CLEAN`
- **Metrics Log**: `LIFEARC_POC.ML_DEMO.MODEL_METRICS_LOG`

### SQL Function for Production Use:
```sql
-- Call from any downstream system
SELECT LIFEARC_POC.ML_DEMO.RESPONSE_CLASSIFIER_CLEAN!PREDICT(
    INPUT_DATA => OBJECT_CONSTRUCT(
        'TRIAL_ID', :trial_id,
        'TREATMENT_ARM', :treatment_arm,
        'BIOMARKER_STATUS', :biomarker_status,
        'CTDNA_CONFIRMATION', :ctdna_confirmation,
        'TARGET_GENE', :target_gene,
        'PATIENT_AGE', :patient_age,
        'PATIENT_SEX', :patient_sex,
        'COHORT', :cohort,
        'BIOMARKER_POSITIVE', :biomarker_positive,
        'CTDNA_CONFIRMED', :ctdna_confirmed,
        'TREATMENT_INTENSITY', :treatment_intensity
    )
):class::INT AS PREDICTION;
```

### Why This Matters for Life Sciences

1. **Data Governance**: All PHI stays in Snowflake
2. **Regulatory Audit**: Full lineage from training data to predictions
3. **Production Ready**: SQL-based inference for any downstream system
4. **Scalable**: Same model works on 1M+ records